# 소환사 닉네임과 태그로 계정 정보 가져오기.

In [64]:
# def league_v4_encryptedSummonerId_info(id):
#     queue = "RANKED_SOLO_5x5"
#     url = f"https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/{id}"
#     response = requests.get(url, headers=header)
    
#     if response.status_code == 200: return response.json()
#     else: print(f"Error fetching data: {response.status_code}"); return None
    

# def account_v1_info(userNickname, tagLine):
#     queue = "RANKED_SOLO_5x5"
#     url = f"https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{userNickname}/{tagLine}"
#     response = requests.get(url, headers=header)
    
#     if response.status_code == 200: return response.json()
#     else: print(f"Error fetching data: {response.status_code}"); return None

# def summoner_v4_by_encryptedPUUID_info(puuid):
#     queue = "RANKED_SOLO_5x5"
#     url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
#     response = requests.get(url, headers=header)
    
#     if response.status_code == 200: return response.json()
#     else: print(f"Error fetching data: {response.status_code}"); return None
    

# player_id = account_v1_info('Justlikethatkr','kr3')
# display(player_id)

# player_id = summoner_v4_by_encryptedPUUID_info(player_id['puuid'])
# display(player_id)

# player = league_v4_encryptedSummonerId_info(player_id['id'])
# display(player)


# Get datas using API

## 챌린저 (300명) 티어 정보 가져오기.

In [ ]:
from config0215 import api_key
import requests
 
api_key = api_key
header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                "Origin": "https://developer.riotgames.com",
                "X-Riot-Token": api_key
                }
 
def league_v4_tier_info(tier): # 챌린저 300명 summonerId 불러오기.
    queue = "RANKED_SOLO_5x5"
    url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    response = requests.get(url, headers=header)
    
    if response.status_code == 200: return response.json()
    else: print(f"Error fetching data: {response.status_code}"); return None
    
    

In [ ]:
tier = 'challenger'
player_json = []

player_json = league_v4_tier_info(tier)['entries']
len(player_json)

300

## 챌린저 (100명) puuid 가져오기. (Match 호출을 위함.)

In [96]:
def summoner_v4_summoners_encryptedSummonerId(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}?api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200 : return response.json()
    else : print(f"Error fetching data: {response.status_code}"); return None

In [ ]:
challenger_play = {}
sorted_entries = sorted(player_json, key=lambda x: x['leaguePoints'], reverse=True)

# puuid 가져오기. (match 데이터를 puuid로 가져올 수 있음.) 최대 2분에 100번 api 호출 가능하기 때문에 100명만 기록.
for index, entry in enumerate(sorted_entries[:100], start=1):
    challenger_play[index] = {
        "puuid" : summoner_v4_summoners_encryptedSummonerId(entry['summonerId'])['puuid'],
        "summonerId": entry["summonerId"],
        "LP": entry["leaguePoints"],
        "wins": entry["wins"],
        "losses": entry["losses"]
    }

print((challenger_play)[1])
print(len(challenger_play))

{'puuid': '0RrhGBQ4ePEl6i7hKVQFRV9NL27lbdOKQCVOHcXWRPeaYqPVE9bM6aLeXUITCHblUmoRBUEhvbJaxw', 'summonerId': 'dF9F3NRdYNBKrIwOMMAX3lYTeMOmvM5hRInM5UAALO4DL2U', 'LP': 1997, 'wins': 177, 'losses': 116}
100


## MatchId 정보 가져오기.
한 명의 유저당 최근 20개의 경기.

In [ ]:
def match_v5_matches_bypuuid_puuid_ids(puuid):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200 : return response.json()
    else : print(f"Error fetching data: {response.status_code}"); return None

In [124]:
matches = []
for player in challenger_play.values():
    matches.extend(match_v5_matches_bypuuid_puuid_ids(player['puuid']))
    
print(len(matches))

2000


## 2000개 경기 정보 가져오기.
1인당 20개씩 총 2000개.

In [144]:
def match_v5_matches_matchid(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200 : return response.json()
    else : print(f'Error fetching data : {response.status_code}'); return 'error'

In [145]:
import pandas as pd
import time

result_list = []

for i, match in enumerate(matches):
    if i % 100 == 0 and i != 0 : print(len(result_list)); time.sleep(121)
    m = match_v5_matches_matchid(match)['info']['participants']
    if m != 'error':
        data = pd.DataFrame(m)
        result_list.append(data)

len(result_list)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


2000

In [147]:
df2 = pd.concat(result_list, ignore_index = True)
df2

,PlayerScore0,PlayerScore1,PlayerScore10,PlayerScore11,PlayerScore2,PlayerScore3,PlayerScore4,PlayerScore5,PlayerScore6,PlayerScore7,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,5,5,0,0,42,10,9,25,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,5,0,0,45,6,3,8,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,4,5,0,0,30,4,5,9,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,5,0,0,58,4,9,25,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,4,5,0,0,130,22,13,73,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,5,0,0,46,3,4,19,True
19996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,2,5,0,0,47,4,13,7,True
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,6,5,0,0,18,3,3,7,True
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,6,5,0,0,45,6,11,16,True


In [148]:
df2.to_csv('matches_2000.csv', index=False)

In [150]:
df2.columns

Index(['PlayerScore0', 'PlayerScore1', 'PlayerScore10', 'PlayerScore11',
       'PlayerScore2', 'PlayerScore3', 'PlayerScore4', 'PlayerScore5',
       'PlayerScore6', 'PlayerScore7',
       ...
       'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills',
       'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame',
       'wardsKilled', 'wardsPlaced', 'win'],
      dtype='object', length=145)

# DATA LOAD

## Data Structure 파악하기.

In [2]:
import pandas as pd

df = pd.read_csv('matches_2000.csv')
df

,PlayerScore0,PlayerScore1,PlayerScore10,PlayerScore11,PlayerScore2,PlayerScore3,PlayerScore4,PlayerScore5,PlayerScore6,PlayerScore7,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,5,5,0,0,42,10,9,25,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,5,0,0,45,6,3,8,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,4,5,0,0,30,4,5,9,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,5,0,0,58,4,9,25,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,4,5,0,0,130,22,13,73,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,5,0,0,46,3,4,19,True
19996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,2,5,0,0,47,4,13,7,True
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,6,5,0,0,18,3,3,7,True
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,6,5,0,0,45,6,11,16,True


In [3]:
cols_bool = df.select_dtypes(include = 'bool').columns
cols_object = df.select_dtypes(include = 'O').columns
cols_num = df.select_dtypes(exclude = 'O').columns
len(cols_bool), len(cols_object), len(cols_num)

(9, 12, 133)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 145 entries, PlayerScore0 to win
dtypes: bool(9), float64(13), int64(111), object(12)
memory usage: 20.9+ MB


In [5]:
df[cols_object].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   challenges          19390 non-null  object
 1   championName        20000 non-null  object
 2   individualPosition  20000 non-null  object
 3   lane                20000 non-null  object
 4   missions            20000 non-null  object
 5   perks               20000 non-null  object
 6   puuid               20000 non-null  object
 7   riotIdGameName      20000 non-null  object
 8   riotIdTagline       20000 non-null  object
 9   role                20000 non-null  object
 10  summonerId          20000 non-null  object
 11  teamPosition        18495 non-null  object
dtypes: object(12)
memory usage: 1.8+ MB


In [6]:
df.describe(include = 'O')

,challenges,championName,individualPosition,lane,missions,perks,puuid,riotIdGameName,riotIdTagline,role,summonerId,teamPosition
count,19390,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,18495
unique,12220,170,6,5,1,12799,2826,2820,1227,5,2826,5
top,"{'12AssistStreakCount': 0, 'HealFromMapSources...",Ezreal,JUNGLE,NONE,"{'playerScore0': 0, 'playerScore1': 0, 'player...","{'statPerks': {'defense': 5011, 'flex': 5008, ...",cduU6SCOLexhaOhH4OPdCyJK6o6qJUGN_ByvBBcX2WjB4p...,Parukia,KR1,SUPPORT,tNL8X41ZMmu4w1dEp8v8GUJkC8zUTcCqozqQVBxijdy6bT...,MIDDLE
freq,9,829,4082,5660,20000,9,141,141,5235,8598,141,3700


In [7]:
df.teamPosition.unique()

array(['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY', nan], dtype=object)

In [8]:
display(df.teamPosition.value_counts().to_frame())

,count
teamPosition,
MIDDLE,3700
UTILITY,3700
BOTTOM,3699
TOP,3698
JUNGLE,3698


# Preprocessing

## 안 쓰는 컬럼 제거

In [9]:
print(cols_num)
df.drop(columns = [col for col in df.columns if col.startswith('PlayerScore')], inplace=True)

Index(['PlayerScore0', 'PlayerScore1', 'PlayerScore10', 'PlayerScore11',
       'PlayerScore2', 'PlayerScore3', 'PlayerScore4', 'PlayerScore5',
       'PlayerScore6', 'PlayerScore7',
       ...
       'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills',
       'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame',
       'wardsKilled', 'wardsPlaced', 'win'],
      dtype='object', length=133)


In [10]:
df.drop('summonerName', axis = 1, inplace=True)

In [11]:
# object type의 columns 중 id 등 쓰지 않는 columns 따로 빼둠.
drop_id_info = df.drop(columns = ['puuid', 'riotIdGameName','riotIdTagline','summonerId'], axis = 1, inplace = True)

In [12]:
df.shape

(20000, 128)

In [13]:
cols_bool = df.select_dtypes(include = 'bool').columns
cols_object = df.select_dtypes(include = 'O').columns
cols_num = df.select_dtypes(include = 'int64').columns
len(cols_bool), len(cols_object), len(cols_num)

(9, 8, 111)

## Information of Each Data Type

In [14]:
# Number Type
pd.set_option('display.max_info_columns', 200)
df[cols_num].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 111 columns):
 #    Column                          Non-Null Count  Dtype
---   ------                          --------------  -----
 0    allInPings                      20000 non-null  int64
 1    assistMePings                   20000 non-null  int64
 2    assists                         20000 non-null  int64
 3    baronKills                      20000 non-null  int64
 4    basicPings                      20000 non-null  int64
 5    bountyLevel                     20000 non-null  int64
 6    champExperience                 20000 non-null  int64
 7    champLevel                      20000 non-null  int64
 8    championId                      20000 non-null  int64
 9    championTransform               20000 non-null  int64
 10   commandPings                    20000 non-null  int64
 11   consumablesPurchased            20000 non-null  int64
 12   damageDealtToBuildings          20000 non-nu

In [15]:
# Object Type
df[cols_object].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   challenges          19390 non-null  object
 1   championName        20000 non-null  object
 2   individualPosition  20000 non-null  object
 3   lane                20000 non-null  object
 4   missions            20000 non-null  object
 5   perks               20000 non-null  object
 6   role                20000 non-null  object
 7   teamPosition        18495 non-null  object
dtypes: object(8)
memory usage: 1.2+ MB


In [16]:
# Boolean Type
df[cols_bool].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   eligibleForProgression     20000 non-null  bool 
 1   firstBloodAssist           20000 non-null  bool 
 2   firstBloodKill             20000 non-null  bool 
 3   firstTowerAssist           20000 non-null  bool 
 4   firstTowerKill             20000 non-null  bool 
 5   gameEndedInEarlySurrender  20000 non-null  bool 
 6   gameEndedInSurrender       20000 non-null  bool 
 7   teamEarlySurrendered       20000 non-null  bool 
 8   win                        20000 non-null  bool 
dtypes: bool(9)
memory usage: 175.9 KB


In [17]:
df.isna().sum().sort_values()

allInPings             0
summoner1Casts         0
subteamPlacement       0
spell4Casts            0
spell3Casts            0
                    ... 
item4                  0
inhibitorsLost         0
win                    0
challenges           610
teamPosition        1505
Length: 128, dtype: int64

## 유의미한 Columns 추출.

In [18]:
# Number, Boolean Type
df_corr_win = df.corr(numeric_only=True).win.abs().sort_values(ascending = False)

display(df_corr_win)
print(sum(df_corr_win > 0.3) - 1) # win 자기 자신 제외한 개수.
useful_cols = df_corr_win[df_corr_win > 0.3].index.tolist()[1:]
useful_cols

win                              1.000000
nexusLost                        0.986839
turretsLost                      0.625210
turretTakedowns                  0.512814
inhibitorsLost                   0.469568
                                   ...   
sightWardsBoughtInGame                NaN
subteamPlacement                      NaN
totalAllyJungleMinionsKilled          NaN
totalEnemyJungleMinionsKilled         NaN
visionClearedPings                    NaN
Name: win, Length: 120, dtype: float64

11


['nexusLost',
 'turretsLost',
 'turretTakedowns',
 'inhibitorsLost',
 'nexusTakedowns',
 'inhibitorTakedowns',
 'deaths',
 'damageDealtToBuildings',
 'damageDealtToTurrets',
 'turretKills',
 'largestKillingSpree']

In [19]:
cols_object

Index(['challenges', 'championName', 'individualPosition', 'lane', 'missions',
       'perks', 'role', 'teamPosition'],
      dtype='object')

In [20]:
useful_cols.extend(['championName', 'individualPosition', 'lane', 'role', 'teamPosition'])
useful_cols

['nexusLost',
 'turretsLost',
 'turretTakedowns',
 'inhibitorsLost',
 'nexusTakedowns',
 'inhibitorTakedowns',
 'deaths',
 'damageDealtToBuildings',
 'damageDealtToTurrets',
 'turretKills',
 'largestKillingSpree',
 'championName',
 'individualPosition',
 'lane',
 'role',
 'teamPosition']

## DTO(challenges)가 Null인 행 drop.

In [21]:
display(df.describe(include = 'O'))
display(len(df.loc[~df.challenges.isnull()]))
# challenges 컬럼에 null 값이 610개 있음. => 제거 필요. 이때, win 비율에 영향을 안 주는지 확인해야함.

,challenges,championName,individualPosition,lane,missions,perks,role,teamPosition
count,19390,20000,20000,20000,20000,20000,20000,18495
unique,12220,170,6,5,1,12799,5,5
top,"{'12AssistStreakCount': 0, 'HealFromMapSources...",Ezreal,JUNGLE,NONE,"{'playerScore0': 0, 'playerScore1': 0, 'player...","{'statPerks': {'defense': 5011, 'flex': 5008, ...",SUPPORT,MIDDLE
freq,9,829,4082,5660,20000,9,8598,3700


19390

In [22]:
df.win.value_counts()

win
True     10000
False    10000
Name: count, dtype: int64

In [23]:
df.loc[~df.challenges.isnull()].win.value_counts()
# challenges의 null 행을 제거해도 win의 T/F 개수가 같으므로 제거 가능.

win
True     9695
False    9695
Name: count, dtype: int64

In [44]:
new_df = df.loc[~df.challenges.isnull()].reset_index(drop=True)
new_df.shape

(19390, 128)

In [46]:
new_df.describe(include ='O')

,challenges,championName,individualPosition,lane,missions,perks,role,teamPosition
count,19390,19390,19390,19390,19390,19390,19390,18495
unique,12220,170,6,5,1,12219,5,5
top,"{'12AssistStreakCount': 0, 'HealFromMapSources...",Ezreal,JUNGLE,NONE,"{'playerScore0': 0, 'playerScore1': 0, 'player...","{'statPerks': {'defense': 5011, 'flex': 5008, ...",SUPPORT,MIDDLE
freq,9,824,4082,5180,19390,9,8090,3700


## DTO data (Challenges, perks) 분리.
missions은 모두 같은 값으로 무의미함. 따라서 사용하지 않음.

In [47]:
import ast

In [48]:
# Challenges
data_list = [ast.literal_eval(d) for d in new_df.challenges]
df_challenges = pd.DataFrame(data_list)
df_challenges.shape

(19390, 141)

In [49]:
# Perks
data_list = [ast.literal_eval(d) for d in new_df.perks]
df_perks = pd.DataFrame(data_list)
df_perks.shape

(19390, 2)

In [50]:
df_challenges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19390 entries, 0 to 19389
Data columns (total 141 columns):
 #    Column                                     Non-Null Count  Dtype  
---   ------                                     --------------  -----  
 0    12AssistStreakCount                        19390 non-null  int64  
 1    HealFromMapSources                         19390 non-null  float64
 2    InfernalScalePickup                        19390 non-null  int64  
 3    SWARM_DefeatAatrox                         19390 non-null  int64  
 4    SWARM_DefeatBriar                          19390 non-null  int64  
 5    SWARM_DefeatMiniBosses                     19390 non-null  int64  
 6    SWARM_EvolveWeapon                         19390 non-null  int64  
 7    SWARM_Have3Passives                        19390 non-null  int64  
 8    SWARM_KillEnemy                            19390 non-null  int64  
 9    SWARM_PickupGold                           19390 non-null  int64  
 10   SWARM_Re

In [51]:
# 유일한 object type 출력.
df_challenges.legendaryItemUsed

0              [6631, 3071, 6333, 3026, 3143]
1              [6692, 3071, 6333, 3026, 3050]
2                    [3118, 4645, 3135, 3089]
3        [6676, 3094, 3036, 3033, 3031, 6675]
4                          [3869, 3190, 3050]
                         ...                 
19385                      [6653, 8010, 3157]
19386          [2503, 4645, 3135, 3118, 6653]
19387          [3153, 6673, 3031, 3139, 3181]
19388                [3153, 3124, 3072, 3139]
19389                      [3869, 3190, 3107]
Name: legendaryItemUsed, Length: 19390, dtype: object

### 유의미한 Columns 추출.

In [52]:
df_challenges.shape, new_df.shape

((19390, 141), (19390, 128))

In [53]:
y = new_df.win
y.shape

(19390,)

In [54]:
df_challenges = pd.concat([df_challenges, y], axis=1)
df_challenges.shape

(19390, 142)

In [67]:
df_challenges_corr = df_challenges.corr(numeric_only=True)['win'].abs().sort_values(ascending = False)[1:]
useful_cols_in_challenges = df_challenges_corr.loc[df_challenges_corr > 0.3].index.tolist()
useful_cols_in_challenges

['kda',
 'turretTakedowns',
 'maxKillDeficit',
 'firstTurretKilled',
 'maxLevelLeadLaneOpponent',
 'deathsByEnemyChamps',
 'takedowns',
 'goldPerMinute',
 'flawlessAces',
 'teamBaronKills',
 'teamRiftHeraldKills']

In [68]:
useful_cols_in_challenges.append('legendaryItemUsed')
useful_cols_in_challenges

['kda',
 'turretTakedowns',
 'maxKillDeficit',
 'firstTurretKilled',
 'maxLevelLeadLaneOpponent',
 'deathsByEnemyChamps',
 'takedowns',
 'goldPerMinute',
 'flawlessAces',
 'teamBaronKills',
 'teamRiftHeraldKills',
 'legendaryItemUsed']

### Create "useful_df" (useful_cols + useful_cols_in_challenges).

In [71]:
len(useful_cols), len(useful_cols_in_challenges)

(16, 12)

In [70]:
useful_df = pd.concat([new_df[useful_cols], df_challenges[useful_cols_in_challenges]], axis = 1)
useful_df.shape

(19390, 28)

In [72]:
useful_df.columns

Index(['nexusLost', 'turretsLost', 'turretTakedowns', 'inhibitorsLost',
       'nexusTakedowns', 'inhibitorTakedowns', 'deaths',
       'damageDealtToBuildings', 'damageDealtToTurrets', 'turretKills',
       'largestKillingSpree', 'championName', 'individualPosition', 'lane',
       'role', 'teamPosition', 'kda', 'turretTakedowns', 'maxKillDeficit',
       'firstTurretKilled', 'maxLevelLeadLaneOpponent', 'deathsByEnemyChamps',
       'takedowns', 'goldPerMinute', 'flawlessAces', 'teamBaronKills',
       'teamRiftHeraldKills', 'legendaryItemUsed'],
      dtype='object')

# Store Data (useful_df, perks_df)

In [78]:
final_df = pd.concat([useful_df, y], axis = 1)
final_df.shape

(19390, 29)

In [81]:
final_df.to_csv('useful_df_except_perks.csv', index=False)

In [82]:
df_perks.to_csv('perks.csv', index=False)

In [83]:
# Check
u_df = pd.read_csv('useful_df_except_perks.csv')
p_df = pd.read_csv('perks.csv')

u_df.shape, p_df.shape

((19390, 29), (19390, 2))

In [84]:
u_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19390 entries, 0 to 19389
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   nexusLost                 19390 non-null  int64  
 1   turretsLost               19390 non-null  int64  
 2   turretTakedowns           19390 non-null  int64  
 3   inhibitorsLost            19390 non-null  int64  
 4   nexusTakedowns            19390 non-null  int64  
 5   inhibitorTakedowns        19390 non-null  int64  
 6   deaths                    19390 non-null  int64  
 7   damageDealtToBuildings    19390 non-null  int64  
 8   damageDealtToTurrets      19390 non-null  int64  
 9   turretKills               19390 non-null  int64  
 10  largestKillingSpree       19390 non-null  int64  
 11  championName              19390 non-null  object 
 12  individualPosition        19390 non-null  object 
 13  lane                      19390 non-null  object 
 14  role  

In [85]:
p_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19390 entries, 0 to 19389
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statPerks  19390 non-null  object
 1   styles     19390 non-null  object
dtypes: object(2)
memory usage: 303.1+ KB


# Perks_df 추출.